In [1]:
import pyranges as pr
import pandas as pd
import numpy as np


# approach

### Input last exons - selecting representative PAS/last exon:

le_ids can have multiple predictions, need to select a single one.
Strategy:
(starting with novel last exons used for quantification) 
1. Prefer events that match atlas
2. if multiple, select min atlas distance
3. If matches, s





## To-dos:
- fix updating to atlas for -ve strand matching pas
- run selection for all novel 3'UTR extensions
- Implement selection for annotated proximal sites of 3'UTR extensions (i.e. most distal annotated site)


# 

In [2]:
# input novel last exons used to generate combined reference of last exons
novel_le = pr.read_gtf("../data/papa/2023-03-29_papa_i3_cortical_upf1_zanovello_overlap_annotated.gtf")
# dexseq results df (used to extract cryptic events of each class)
dexseq_df = pd.read_csv("../data/papa/2023-05-24_i3_cortical_zanovello.all_datasets.dexseq_apa.results.processed.cleaned.tsv", sep="\t")
# refeence GTF used to identify novel last exons, quantify vs ref
ref_gtf = pr.read_gtf("../data/reference_filtered.gtf")
# 
tx2le = pd.read_csv("../data/papa/novel_ref_combined.tx2le.tsv", sep="\t")

# last exon quantification regions used as input to Salmon
quant_uniq_le = pr.read_gtf("../data/papa/novel_ref_combined.quant.last_exons.gtf")


info_df = pd.read_csv("../data/papa/novel_ref_combined.info.tsv", sep="\t")


In [3]:
print(quant_uniq_le.columns)
quant_uniq_le.subset(lambda df: ~df.transcript_id.str.startswith("ENST"))

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_name', 'transcript_id', 'exon_number',
       'region_rank', 'Start_ref', 'End_ref', 'transcript_id_ref',
       '3p_extension_length', 'event_type', 'ref_gene_id', 'ref_gene_name',
       'le_number', 'le_id'],
      dtype='object')


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24476667,.,+,.,PAPA.NT_19074713_S59.398,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
1,chr1,.,exon,24468985,24476667,.,+,.,PAPA.ctrl_ctrl_3.424,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
2,chr1,.,exon,24468985,24475256,.,+,.,PAPA.TDP43_19065409_S29.438,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
3,chr1,.,exon,24468985,24473773,.,+,.,PAPA.Cont-B_S2.334,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
4,chr1,.,exon,24468985,24474852,.,+,.,PAPA.DZ_curves_1_3.360,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36889,chrY,.,exon,13234576,13234826,.,-,.,PAPA.NT_19074729_S12.25635,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36890,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43-F_S6.20683,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36891,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43_19065413_S19.26168,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36892,chrY,.,exon,13234583,13234826,.,-,.,PAPA.DZ_curves_0_2.21889,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7


In [4]:
dexseq_df.columns

Index(['experiment_name', 'binID', 'groupID', 'featureID', 'exonBaseMean',
       'dispersion', 'stat', 'pvalue', 'padj', 'UsageCoefficient_base',
       'UsageCoefficient_contrast', 'log2fold_contrast_base', 'le_id',
       'gene.qvalue', 'contrast_name', 'gene_id', 'gene_name', 'event_type',
       'annot_status', 'transcript_id', 'chromosome', 'strand', 'start', 'end',
       'mean_PPAU_base', 'mean_PPAU_treatment', 'delta_PPAU_treatment_control',
       'simple_event_type'],
      dtype='object')

In [5]:
novel_le.columns

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'transcript_id', 'event_type',
       '3p_extension_length', 'atlas_filter', 'nearest_atlas_distance', 'Name',
       'motif_filter', 'pas_motifs', 'condition_id', 'le_id', 'gene_name_ref',
       'gene_id_ref', 'mean_PPAU_base', 'mean_PPAU_treatment',
       'delta_PPAU_treatment_control', 'experiment_id', 'Cluster',
       'experiment_count', 'multiple_datasets', 'max_mean_PPAU'],
      dtype='object')

In [12]:
# get le_ids that are novel 3'UTR ectensions
le_id_d3utr = set(dexseq_df[dexseq_df.simple_event_type == "distal_3utr_extension"].le_id)
len(le_id_d3utr)

3369

In [126]:
def n_uniq_coords(df):

    if df["Strand"].unique()[0] == "+":
        n_coords = df["End"].nunique()
    
    else:
        n_coords = df["Start"].nunique()

    return n_coords


def select_rep_site(df):
# First select sites of le_id that match atlas site (if applicable)
    if df["atlas_filter"].astype(int).sum() > 0:

        out_df = df[df["atlas_filter"].eq("1")]

        # 2. Within these, see if multiple different 3'ends predicted
        ### TODO: use Name (atlas ID) instead of 3' coordinate to match
        n_atlas = out_df["Name"].nunique()

        if n_atlas > 1:
                # multiple different atlas sites predicted
                # see if somes sites were called in > 1 dataset - select those with max number of datasets called
                end3_c = out_df.groupby("Name")["experiment_id"].nunique()
                # have a series with Name (PolyASite ID) as index labels, count of experimnts as values
                # get the IDs with largest number of unqiue experiments (as as series)
                end3_c = end3_c[end3_c == end3_c.max()]

                if len(end3_c) > 1:
                    # multiple 3'ends/ atlas sites predicted in same number of datasets
                    # raise Exception(f"{out_df['le_id_quant'].drop_duplicates().iloc[0]} - le_id has multiple predicted atlas sites")
                    # prefer the shortest extension in this case (conservative, defines smaller window in)

                    # calculate length
                    out_df = out_df[out_df["Name"].isin(end3_c.index)]
                    out_df["reg_len"] = out_df.End - out_df.Start

                    # select shortest extension
                    out_df = out_df[out_df.reg_len == out_df.reg_len.min()]
                    # Select a single row for le_id corresponding to selected PAS
                    out_df = out_df.sort_values(by="experiment_id").drop_duplicates(subset=["Start", "End", "Strand"]).drop(columns="reg_len")

                    return out_df, "atlas_max_datasets_shortest"
                                    
                else:
                    # 1 pas remaining
                    end3_coord = end3_c.index[0]

                    # Select a single row for le_id corresponding to selected PAS
                    out_df = out_df[out_df["Name"] == end3_coord].sort_values(by="experiment_id").drop_duplicates(subset=["Start", "End", "Strand"])

                    decision_str = "atlas_max_datasets"


        else:
            # only single atlas site predicted - just select a single row
            end3_coord = out_df["Name"].drop_duplicates().index[0]
            # Select a single row for le_id corresponding to selected PAS
            out_df = out_df[out_df["Name"] == end3_coord].sort_values(by="experiment_id").drop_duplicates(subset=["Start", "End", "Strand"])

            decision_str = "atlas_1_pred"
        

        return out_df, decision_str

    else:
        # TODO: implement filtering for events called with motifs only
        out_df = df[df["atlas_filter"].eq("0")]

        # select predicted site with motif that is closest to expected distance (21nt upstream)
        # -74_TATAAA,-14_TATAAA
        # [-74_TATAAA, -14_TATAAA]
        # [-74, -14]
        # absolute distance from expected position (21nt upstream)
        # select min deviating motif for each predicted PAS
        pas_m = out_df["pas_motifs"].apply(lambda x: min([abs(21 - abs(int(i.split("_")[0]))) for i in x.split(",")]))

        # select predicted pas with minimum deviation from 21nt
        pas_m = pas_m[pas_m == pas_m.min()]

        if len(pas_m) > 1:
            # multiple predicted PAS with motif same distance from expected

            # check if exactly the same coordinate predicted for these events

            # (if true, just sort alphabetically by experiment id and pick first)
            # otherwise, select shortest
            n_pas = n_uniq_coords(out_df)

            if n_pas > 1:
                # select shortest extension
                # calculate length
                out_df = out_df.loc[pas_m.index, :]

                out_df["reg_len"] = out_df.End - out_df.Start

                # select shortest extension
                out_df = out_df[out_df.reg_len == out_df.reg_len.min()]
                # Select a single row for le_id corresponding to selected PAS
                out_df = out_df.sort_values(by="experiment_id").drop_duplicates(subset=["Start", "End", "Strand"]).drop(columns="reg_len")

            else:
                # just a single coord predicted in multiple datasets - 
                out_df = out_df.sort_values(by="experiment_id").drop_duplicates(subset=["Start", "End", "Strand"])

            return out_df, "motif_shortest_min"
        
        else:
            out_df = out_df.loc[pas_m.index, :]
            

            decision_str = "motif_1_min"


        return out_df, decision_str


    
    

In [130]:
#### SELECTING REPRESENTATIVE PAS
# LEs have to be quantified
novel_le_df = novel_le.apply(lambda df: df.merge(tx2le, on="transcript_id", how="inner", suffixes=[None, "_quant"])).as_df()


i = 0

sel_dfs = []
sel_dict = {"atlas_1_pred": [],
            "atlas_max_datasets": [],
            "atlas_max_datasets_shortest": [],
            "motif_1_min": [],
            "motif_shortest_min": []}

le_grpd = iter(novel_le_df[novel_le_df.le_id.isin(le_id_d3utr)].groupby("le_id_quant"))



while i < 100:
        le_id, df = next(le_grpd)
        # print(le_id)
        out_df, decision_str = select_rep_site(df)
        sel_dfs.append(out_df)
        sel_dict[decision_str].append(le_id)
        i +=1  



sel_novel_le_df = pd.concat(sel_dfs)               
    
print({dec: len(ids) for dec, ids in sel_dict.items()})


{'atlas_1_pred': 41, 'atlas_max_datasets': 20, 'atlas_max_datasets_shortest': 8, 'motif_1_min': 29, 'motif_shortest_min': 2}


In [120]:
novel_le_df[novel_le_df["le_id_quant"].isin(sel_dict["atlas_multiple_sites"])][["le_id_quant", "experiment_id", "Name", "Start", "End", "experiment_count", "transcript_id", "Strand"]].loc[lambda df: df.le_id_quant == ]

,le_id_quant,experiment_id,Name,Start,End,experiment_count,transcript_id,Strand
33375,ENSG00000006757.12_2,zanovello_i3_cortical_upf1_tdp_tdpkd_upf1ctl_v...,X:7897884:-,7897883,7900817,2,PAPA.ctrl_ctrl_2.26964.1,-
33376,ENSG00000006757.12_2,zanovello_i3_cortical_upf1_tdp_tdpkd_upf1ctl_v...,X:7897884:-,7897883,7900817,2,PAPA.TDP43_ctrl_4.26054.2,-
33377,ENSG00000006757.12_2,seddighi_i3_cortical,X:7898247:-,7898079,7900817,2,PAPA.NT_19074717_S21.25514.1,-
33378,ENSG00000006757.12_2,zanovello_i3_cortical_upf1_tdp_tdpkd_upf1ctl_v...,X:7898247:-,7898128,7900817,2,PAPA.ctrl_ctrl_4.26703.2,-
33379,ENSG00000006757.12_2,seddighi_i3_cortical,X:7898247:-,7898132,7900817,2,PAPA.TDP43_19065403_S23.25616.1,-
33380,ENSG00000006757.12_2,zanovello_i3_cortical_upf1_tdp_tdpkd_upf1ctl_v...,X:7898247:-,7898136,7900817,2,PAPA.TDP43_ctrl_1.25803.1,-
33381,ENSG00000006757.12_2,seddighi_i3_cortical,X:7898247:-,7898241,7900817,2,PAPA.NT_19074721_S50.25909.1,-


In [129]:
sel_novel_le_df[sel_novel_le_df["le_id_quant"].isin(sel_dict["atlas_max_datasets_shortest"])][["le_id_quant", "experiment_id", "Name", "Start", "End", "experiment_count", "transcript_id", "Strand"]]
#.loc[lambda df: df.le_id_quant == "ENSG00000006757.12_2", :]

,le_id_quant,experiment_id,Name,Start,End,experiment_count,transcript_id,Strand
13794,ENSG00000001629.10_3,zanovello_i3_cortical_upf1_tdp_tdpkd_upf1ctl_v...,7:92401774:+,92398211,92401760,6,PAPA.TDP43_ctrl_2.10802.2,+
14443,ENSG00000005108.17_3,zanovello_shsy5y_curve_0075,7:11370437:-,11370452,11375878,7,PAPA.doxconc_NT_0_2.7361.1,-
14450,ENSG00000006468.14_4,humphrey_i3_cortical,7:13890964:-,13890946,13896087,3,PAPA.TDP-3.9905.1,-
33381,ENSG00000006757.12_2,seddighi_i3_cortical,X:7898247:-,7898241,7900817,2,PAPA.NT_19074721_S50.25909.1,-
13230,ENSG00000008018.9_2,seddighi_i3_cortical,6:170534538:-,170534537,170535405,3,PAPA.TDP43_19065409_S29.9994.1,-
32454,ENSG00000015475.19_2,humphrey_i3_cortical,22:17733561:-,17733560,17735591,4,PAPA.CTRL-5.21657.1,-
21869,ENSG00000018236.15_4,brown_i3_cortical,12:41073374:+,41069958,41073383,6,PAPA.TDP43-F_S6.12521.4,+
764,ENSG00000031698.13_3,zanovello_shsy5y_chx_kd_only,1:109238791:+,109237730,109238792,2,PAPA.chx_tdp_CTRL_ctrl_4.997.3,+


In [ ]:
print(novel_le_df[novel_le_df["le_id_quant"] == "ENSG00000001631.17_3"].columns)
novel_le_df[novel_le_df["le_id_quant"] == "ENSG00000001631.17_3"][["Start", "End", "atlas_filter", "Name", "le_id", "gene_name_ref"]]

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'transcript_id', 'event_type',
       '3p_extension_length', 'atlas_filter', 'nearest_atlas_distance', 'Name',
       'motif_filter', 'pas_motifs', 'condition_id', 'le_id', 'gene_name_ref',
       'gene_id_ref', 'mean_PPAU_base', 'mean_PPAU_treatment',
       'delta_PPAU_treatment_control', 'experiment_id', 'Cluster',
       'experiment_count', 'multiple_datasets', 'max_mean_PPAU',
       'le_id_quant'],
      dtype='object')


,Start,End,atlas_filter,Name,le_id,gene_name_ref
14815,92198195,92200804,0,7:92197498:-,ENSG00000001631.17_3,KRIT1
14816,92198602,92200804,0,7:92199243:-,ENSG00000001631.17_3,KRIT1
14817,92199253,92200804,1,7:92199243:-,ENSG00000001631.17_3,KRIT1
14818,92199257,92200804,1,7:92199243:-,ENSG00000001631.17_3,KRIT1
